In [21]:
import folium

lat1, long1 = 37.52860, 126.93431 # 여의도 한강공원
lat2, long2 = 37.52400, 126.91889 # 여의도공원
lat3, long3 = 37.51865, 126.92041 # 샛강생태공원


map_y = folium.Map([lat1, long1], zoom_start=15)
folium.Marker([lat1, long1], tooltip='여의도 한강공원', icon=folium.Icon(color='red', icon='automobile', prefix='fa')).add_to(map_y) # 마커 표시하기
folium.Marker([lat2, long2], tooltip='여의도공원', icon=folium.Icon(color='blue', icon='balance-scale', prefix='fa')).add_to(map_y) # 마커 표시하기
folium.Marker([lat3, long3], tooltip='샛강생태공원', icon=folium.Icon(color='purple', icon='ban', prefix='fa')).add_to(map_y) # 마커 표시하기
map_y

In [22]:
import folium

lat = [37.52860, 37.52400, 37.51865]                        # 위도 리스트
long = [126.93431, 126.91889, 126.92041]                    # 경도 리스트
names = ['여의도 한강공원', '여의도공원', '샛강생태공원']   # 위치 이름 리스트
icons = ['automobile', 'balance-scale', 'ban']              # 아이콘 모양 리스트
colors = ['red', 'blue', 'purple']                          # 아이콘 색상 리스트

map_y = folium.Map([lat[0], long[0]], zoom_start=15)
for i in range(len(lat)):
    folium.Marker(
        [lat[i], long[i]], tooltip=names[i],
        icon=folium.Icon(color=colors[i], icon=icons[i], prefix='fa')
        ).add_to(map_y)
map_y

In [8]:
# 인구가 많은 상위 10개 행정동을 뽑아 지도에 표시하기
import csv
import operator

f = open('./data/LOCAL_PEOPLE_DONG_201912.csv', encoding='utf8') # 또는 'eur-kr', 'cp949'
data = csv.reader(f)
next(data)
data = list(data)
print(len(data))

f2 = open('./data/dong_code.csv', encoding = 'cp949')
code_data = csv.reader(f2)
next(code_data)
next(code_data)
code_data = list(code_data)
print(len(code_data))

315456
424


In [12]:
f3 = open('./data/dong_location.csv', encoding = 'cp949')
location_data = csv.reader(f3)
next(location_data)
next(location_data)
location_data = list(location_data)
print(len(location_data))

437


In [ ]:
print(location_data)

In [156]:
# row[2] # 행정동
# row[3] # 총생활인구수
total = {}
dong_code, counts = 0, 0
for row in data:
    dong_code, counts = row[2], float(row[3])
    if dong_code not in total.keys():
        total[dong_code] = counts
    else:
        total[dong_code] += counts
top10 = sorted(total.items(), key=operator.itemgetter(1), reverse=True)[:10] # 밸류값 내림차순 정렬 후 상위 10개 리턴

In [157]:
print(top10)

[('11680640', 83625140.5283), ('11560540', 70145934.57189998), ('11440660', 62110058.62770009), ('11110615', 53902615.507999994), ('11650530', 50479699.55179994), ('11410585', 47971139.75410003), ('11380690', 47402912.95459995), ('11545510', 42542504.264800064), ('11680650', 41410763.110000014), ('11680610', 39427473.14730003)]


In [171]:
# 행정동 코드 교체
top10_change = []
for row in code_data:
    for i in top10:
        if i[0] == row[1]:
            # print(i)
            top10_change.append([i[1], row[0],row[4]])
top10_change.sort(reverse=True)
print(top10_change)

[[83625140.5283, '1123064', '역삼1동'], [70145934.57189998, '1119054', '여의동'], [62110058.62770009, '1114066', '서교동'], [53902615.507999994, '1101061', '종로1.2.3.4가동'], [50479699.55179994, '1122053', '서초3동'], [47971139.75410003, '1113075', '신촌동'], [47402912.95459995, '1112071', '진관동'], [42542504.264800064, '1118051', '가산동'], [41410763.110000014, '1123065', '역삼2동'], [39427473.14730003, '1123079', '대치2동']]


In [208]:
map_info = []
for i in location_data:
    for j in top10_change:
        # print(j[1])
        if i[2] == j[1]: # 조회하고자하는 코드가 같으면
            map_info.append([j[0], i[3], i[-1], i[-2]])
map_info.sort(reverse=True)

import folium

map_y = folium.Map([map_info[0][-2], map_info[0][-1]], zoom_start=12)
for i in range(len(map_info)):
    folium.Marker(
        [map_info[i][-2], map_info[i][-1]], tooltip=(f'{i+1}위, {map_info[i][1]}')).add_to(map_y)
map_y

In [250]:
for row in data:
    for i in range(1,32):
        if i<=2:
            row[i] = int(row[i])
        else:
            row[i] = float(row[i])
print(data[0])

['20191201', 0, 11215710, 45694.9988, 452.3937, 208.7971, 1267.7678, 5642.9235, 5433.6801, 2494.5993, 1691.4831, 1042.7096, 1058.3378, 895.4639, 875.9212, 601.9097, 496.2165, 850.682, 388.4005, 180.3721, 1586.7039, 6973.6176, 4259.7022, 2036.1446, 1285.42, 794.6241, 833.7179, 772.1417, 773.2152, 827.1455, 565.5378, 1405.3707, '']


In [255]:
# 남성이 많은 지역과 여성이 많은 지역 비교하기

male, female = {}, {}
code, male_total, female_total, = 0, 0, 0
for row in data:
    code, male_total, female_total = row[2], row[4:18], row[18:32]
    # 남자 처리
    if code not in male.keys():
        male[code] = sum(male_total)
    else:
        male[code] += sum(male_total)

    # 여자 처리
    if code not in female.keys():
        female[code] = sum(female_total)
    else:
        female[code] += sum(female_total)

male10 = sorted(male.items(), key=operator.itemgetter(1), reverse=True)[:10] # 밸류값 내림차순 정렬 후 상위 10개 리턴
female10 = sorted(female.items(), key=operator.itemgetter(1), reverse=True)[:10] # 밸류값 내림차순 정렬 후 상위 10개 리턴


In [280]:
print(male10)

[(11680640, 42930119.54840002), (11560540, 37888759.144700006), (11110615, 30088470.77510002), (11440660, 27909574.7458), (11650530, 25914922.402299967), (11545510, 24638497.676100004), (11380690, 21529598.94819998), (11680610, 20200091.842699986), (11410585, 19801070.219000008), (11680650, 19748653.11170003)]


In [337]:
# data
# code_data
# location_data
male_map_info, female_map_info = [], []
# 남자 처리
for row in code_data:
    for i in range(len(male10)):
        if male10[i][0] == int(row[1]):
            for loc in location_data:
                if row[4] == loc[3]:
                    male_map_info.append([male10[i][1], loc[3],loc[-1], loc[-2]])
male_map_info.sort(reverse=True)
print(male_map_info)

# 여자 처리
for row in code_data:
    for i in range(len(female10)):
        if female10[i][0] == int(row[1]):
            for loc in location_data:
                if row[4] == loc[3]:
                    female_map_info.append([female10[i][1], loc[3],loc[-1], loc[-2]])
female_map_info.sort(reverse=True)
print(female_map_info)

[[42930119.54840002, '역삼1동', '37.4999211', '127.0367584'], [30088470.77510002, '종로1.2.3.4가동', '37.5777701', '126.9892737'], [27909574.7458, '서교동', '37.5550337', '126.9228533'], [25914922.402299967, '서초3동', '37.4856591', '127.0097616'], [24638497.676100004, '가산동', '37.4756389', '126.8850721'], [21529598.94819998, '진관동', '37.64193', '126.9387277'], [20200091.842699986, '대치2동', '37.4955294', '127.0662436'], [19801070.219000008, '신촌동', '37.5659638', '126.9384315'], [19748653.11170003, '역삼2동', '37.4984733', '127.0449272']]
[[40695020.916000046, '역삼1동', '37.4999211', '127.0367584'], [34200483.915800005, '서교동', '37.5550337', '126.9228533'], [28170069.535899986, '신촌동', '37.5659638', '126.9384315'], [25873314.060800016, '진관동', '37.64193', '126.9387277'], [24564777.121699978, '서초3동', '37.4856591', '127.0097616'], [23814144.75530002, '종로1.2.3.4가동', '37.5777701', '126.9892737'], [21662109.979499962, '역삼2동', '37.4984733', '127.0449272'], [21042903.097300008, '잠실3동', '37.5162319', '127.0947833'], [2

In [359]:

map_y = folium.Map([male_map_info[0][-2], male_map_info[0][-1]], zoom_start=12)
for i in range(len(male_map_info)):
    folium.Marker(
        [male_map_info[i][-2], male_map_info[i][-1]], tooltip=(f'{i+1}위, {male_map_info[i][1]}'), icon=folium.Icon(color='red', icon='male', prefix='fa')).add_to(map_y)
    folium.Marker(
        [female_map_info[i][-2], female_map_info[i][-1]], tooltip=(f'{i+1}위, {female_map_info[i][1]}'), icon=folium.Icon(color='blue', icon='female', prefix='fa')).add_to(map_y)
map_y

In [367]:
import folium

lat = [35.15870, 35.23360, 35.17357]                        # 위도 리스트
long = [129.16047, 129.08309, 128.94632]                    # 경도 리스트
names = ['해운대 해수욕장', '부산대학교', '김해국제공항']   # 위치 이름 리스트
icons = ['hand-o-down', 'university', 'plane']              # 아이콘 모양 리스트
colors = ['red','yellow','blue']                          # 아이콘 색상 리스트

map_y = folium.Map([lat[0], long[0]], zoom_start=15)
for i in range(len(lat)):
    folium.Marker(
        [lat[i], long[i]], tooltip=names[i],
        icon=folium.Icon(color=colors[i], icon=icons[i], prefix='fa')
        ).add_to(map_y)
map_y

C:\Users\Jasper\AppData\Local\Temp\ipykernel_2308\3029650208.py:13: UserWarning: color argument of Icon should be one of: {'darkgreen', 'purple', 'red', 'darkpurple', 'white', 'cadetblue', 'lightgray', 'gray', 'black', 'green', 'darkred', 'lightred', 'darkblue', 'blue', 'lightgreen', 'lightblue', 'pink', 'beige', 'orange'}.
  icon=folium.Icon(color=colors[i], icon=icons[i], prefix='fa')
